# Initialising

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

from numpy import mean
from numpy import std
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
# link to drive
from google.colab import drive
drive.mount('/content/drive')

# sample splitting code

In [ ]:
def initialise(data):
  data.index = data["date"]
  data.index = pd.to_datetime(data.index)
  data.drop(columns=["date"], inplace=True)
  return data

In [ ]:
def samplesplitting(data,train,validate,oos,i):
    train_data    = data[(data.index.year >= 1996 )         & (data.index.year < i - validate)]
    validate_data = data[(data.index.year >= i - validate)  & (data.index.year < i)]
    total_train   = data[(data.index.year >= 1996 )         & (data.index.year < i)]
    oos_data      = data[(data.index.year == i)]
    pred_data     = data[(data.index.year >= 1996 + train + validate) & (data.index.year <= i)]

    print('train=%d, validate=%d, oos=%d' % (len(train_data), len(validate_data), len(oos_data)))
    return train_data, validate_data, oos_data, total_train, pred_data

def split_xy(data, kind):
  data_x = data.iloc[:,8:]
  data_y = data[kind]
  return data_x, data_y

#  Download data

Data standardised with dummies

In [ ]:
data = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_standardized_dum.csv'))
data.head()

,secid,-80,-50,-20,level,slope,curve,permno,mvel1,beta,...,dummy_75.0,dummy_76.0,dummy_78.0,dummy_79.0,dummy_80.0,dummy_81.0,dummy_82.0,dummy_83.0,dummy_87.0,dummy_99.0
date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,5005,0.671170,0.619801,0.617194,0.619801,-0.002607,0.024381,85041,-0.357137,-0.562815,...,-0.041345,0.0,-0.053406,-0.117647,-0.138313,0.0,-0.033748,0.0,-0.079351,-0.075636
1996-01-31,5015,0.289023,0.280235,0.282039,0.280235,0.001804,0.005296,56266,-0.023589,-0.440098,...,-0.041345,0.0,-0.053406,-0.117647,-0.138313,0.0,-0.033748,0.0,-0.079351,-0.075636
1996-01-31,5022,0.945362,0.426352,0.359600,0.426352,-0.066752,0.226129,63845,-0.300442,-0.393064,...,-0.041345,0.0,-0.053406,-0.117647,-0.138313,0.0,-0.033748,0.0,-0.079351,-0.075636
1996-01-31,5028,0.748517,0.719571,0.694680,0.719571,-0.024891,0.002027,12205,-0.367024,2.116755,...,-0.041345,0.0,-0.053406,-0.117647,-0.138313,0.0,-0.033748,0.0,-0.079351,-0.075636
1996-01-31,5029,0.475552,0.475552,0.475552,0.475552,0.000000,0.000000,60046,-0.279931,-0.610756,...,-0.041345,0.0,-0.053406,-0.117647,-0.138313,0.0,-0.033748,0.0,-0.079351,-0.075636


Data standardised without dummies

In [ ]:
data_final = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_final_low.csv'))
data_final.head()

,secid,-80,-50,-20,level,slope,curve,permno,mvel1,beta,...,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2
date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,5005,0.671170,0.619801,0.617194,0.619801,-0.002607,0.024381,85041,379771.5,0.898022,...,0.070022,1.0,0.044382,7.901122e-09,0.068493,0.029538,0.575223,4.348170,1.511698e-08,35.0
1996-01-31,5015,0.289023,0.280235,0.282039,0.280235,0.001804,0.005296,56266,3113273.0,0.978254,...,0.136184,0.0,0.020703,6.933274e-10,0.019231,0.011278,0.479959,1.686139,2.905242e-08,60.0
1996-01-31,5022,0.945362,0.426352,0.359600,0.426352,-0.066752,0.226129,63845,844398.0,1.009004,...,0.303921,3.0,0.029187,1.580689e-08,0.032258,0.018749,0.650900,1.222107,7.321262e-08,64.0
1996-01-31,5028,0.748517,0.719571,0.694680,0.719571,-0.024891,0.002027,12205,298740.0,2.649911,...,0.049865,6.0,0.041775,7.590812e-09,0.071429,0.027339,0.831621,14.193444,8.397239e-09,36.0
1996-01-31,5029,0.475552,0.475552,0.475552,0.475552,0.000000,0.000000,60046,1012486.0,0.866678,...,0.186537,3.0,0.030354,9.242042e-09,0.038168,0.024072,0.713023,2.276398,3.759432e-08,35.0


Data mapped

In [ ]:
data_mapped = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_mapped.csv'))
data_mapped.head()

,secid,-80,-50,-20,level,slope,curve,permno,mvel1,beta,...,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2
date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,5005,0.671170,0.619801,0.617194,0.619801,-0.002607,0.024381,85041,-0.999720,-0.329763,...,-0.999953,-0.75,-0.739932,-0.999861,-0.862116,-0.777854,-0.657852,-0.989057,-1.0,-0.306122
1996-01-31,5015,0.289023,0.280235,0.282039,0.280235,0.001804,0.005296,56266,-0.997704,-0.294073,...,-0.999909,-1.00,-0.878965,-0.999988,-0.964248,-0.915183,-0.714516,-0.995756,-1.0,0.204082
1996-01-31,5022,0.945362,0.426352,0.359600,0.426352,-0.066752,0.226129,63845,-0.999378,-0.280395,...,-0.999797,-0.25,-0.829151,-0.999721,-0.937239,-0.858996,-0.612839,-0.996924,-1.0,0.285714
1996-01-31,5028,0.748517,0.719571,0.694680,0.719571,-0.024891,0.002027,12205,-0.999780,0.449524,...,-0.999967,0.50,-0.755235,-0.999866,-0.856030,-0.794394,-0.505345,-0.964279,-1.0,-0.285714
1996-01-31,5029,0.475552,0.475552,0.475552,0.475552,0.000000,0.000000,60046,-0.999254,-0.343705,...,-0.999876,-0.25,-0.822297,-0.999837,-0.924987,-0.818967,-0.575888,-0.994271,-1.0,-0.306122


Data standardised

In [ ]:
data = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_standardized.csv'))
data.head()

,secid,-80,-50,-20,level,slope,curve,permno,mvel1,beta,...,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2
date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,5005,0.671170,0.619801,0.617194,0.619801,-0.002607,0.024381,85041,-0.217396,-0.541677,...,-0.118045,-2.170908,0.322335,-0.027982,0.196386,0.150430,0.409407,-0.163794,-0.026501,-0.674854
1996-01-31,5015,0.289023,0.280235,0.282039,0.280235,0.001804,0.005296,56266,-0.121171,-0.403972,...,-0.117527,-2.857980,-0.713330,-0.041094,-0.769715,-0.839669,-0.072886,-0.329598,-0.026501,0.660746
1996-01-31,5022,0.945362,0.426352,0.359600,0.426352,-0.066752,0.226129,63845,-0.201040,-0.351195,...,-0.116213,-0.796764,-0.342265,-0.013601,-0.514233,-0.434579,0.792536,-0.358500,-0.026500,0.874442
1996-01-31,5028,0.748517,0.719571,0.694680,0.719571,-0.024891,0.002027,12205,-0.220248,2.465157,...,-0.118203,1.264451,0.208339,-0.028546,0.253954,0.031181,1.707473,0.449418,-0.026501,-0.621430
1996-01-31,5029,0.475552,0.475552,0.475552,0.475552,0.000000,0.000000,60046,-0.195123,-0.595474,...,-0.117132,-0.796764,-0.291208,-0.025543,-0.398332,-0.145984,1.107042,-0.292834,-0.026501,-0.674854


Data standardised per period without SIC

In [ ]:
data = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_standardized_pp_nosic.csv'))
data.head()

,secid,-80,-50,-20,level,slope,curve,permno,mvel1,beta,...,stdacc,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade
date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,5005,0.671170,0.619801,0.617194,0.619801,-0.002607,0.024381,85041,-0.357137,-0.562815,...,-0.096071,-0.095942,-2.185988,0.576182,-0.144109,0.303505,0.266900,-0.378472,-0.130649,-0.033749
1996-01-31,5015,0.289023,0.280235,0.282039,0.280235,0.001804,0.005296,56266,-0.023589,-0.440098,...,-0.085433,-0.089558,-2.857014,-0.654499,-0.463328,-0.873802,-0.911291,-0.874469,-0.508309,-0.033748
1996-01-31,5022,0.945362,0.426352,0.359600,0.426352,-0.066752,0.226129,63845,-0.300442,-0.393064,...,-0.055550,-0.073373,-0.843936,-0.213563,0.206021,-0.562466,-0.429244,0.015543,-0.574140,-0.033747
1996-01-31,5028,0.748517,0.719571,0.694680,0.719571,-0.024891,0.002027,12205,-0.367024,2.116755,...,-0.102722,-0.097887,1.169143,0.440721,-0.157852,0.373658,0.124998,0.956476,1.266088,-0.033749
1996-01-31,5029,0.475552,0.475552,0.475552,0.475552,0.000000,0.000000,60046,-0.279931,-0.610756,...,-0.061928,-0.084700,-0.843936,-0.152891,-0.084723,-0.421228,-0.085824,0.338985,-0.424569,-0.033748


Standardised data with selected stocks

In [7]:
data = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_standardized_selection10.csv'))
data.head()

,secid,-80,-50,-20,level,slope,curve,permno,mvel1,beta,...,stdacc,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade
date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,100862,0.307688,0.315627,0.358095,0.315627,0.042468,0.017264,54594,-0.406584,-0.639001,...,-0.089312,-0.081404,-0.324628,-0.424243,1.466355,0.383028,-0.139901,2.271301,-0.457693,-0.047324
1996-01-31,100871,0.705411,0.615432,0.651160,0.615432,0.035728,0.062854,50906,-0.214282,2.183274,...,-0.097775,-0.084204,1.735954,1.474995,-0.398712,1.612154,1.173917,0.136424,0.297064,-0.047325
1996-01-31,100892,0.270090,0.216768,0.216508,0.216768,-0.000260,0.026531,57904,-0.032587,-0.731705,...,-0.081186,-0.077051,0.362233,-0.746485,-0.409080,-0.900621,-0.994256,-0.481665,-0.534902,-0.047325
1996-01-31,100896,0.430796,0.436291,0.429517,0.436291,-0.006774,-0.006135,77520,-0.321765,1.642115,...,0.196797,0.040956,-1.011489,-0.617592,-0.389601,0.442328,-0.418152,0.171988,0.018494,-0.047325
1996-01-31,100903,0.262405,0.252683,0.373839,0.252683,0.121156,0.065439,80303,-0.352200,0.206117,...,-0.081186,-0.077051,-1.698350,-0.815089,-0.233912,-0.351483,-0.529485,-0.049179,-0.174634,-0.047325


In [11]:
data = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_mapped_selection10.csv'))
data.head()

,secid,-80,-50,-20,level,slope,curve,permno,mvel1,beta,...,stdacc,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade
date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,100862,0.307688,0.315627,0.358095,0.315627,0.042468,0.017264,54594,-0.994821,-0.502126,...,-0.997784,-0.999126,0.00,-0.689049,-0.775188,-0.662673,-0.619551,0.327234,-0.936622,-1.0
1996-01-31,100871,0.705411,0.615432,0.651160,0.615432,0.035728,0.062854,50906,-0.962567,0.551365,...,-0.998741,-0.999446,0.75,-0.112575,-0.989945,-0.390130,-0.223204,-0.342296,-0.748948,-1.0
1996-01-31,100892,0.270090,0.216768,0.216508,0.216768,-0.000260,0.026531,57904,-0.932091,-0.536730,...,-0.996865,-0.998628,0.25,-0.786859,-0.991139,-0.947305,-0.877289,-0.536138,-0.955821,-1.0
1996-01-31,100896,0.430796,0.436291,0.429517,0.436291,-0.006774,-0.006135,77520,-0.980594,0.349363,...,-0.965422,-0.985129,-0.25,-0.747736,-0.988896,-0.649524,-0.703492,-0.331142,-0.818216,-1.0
1996-01-31,100903,0.262405,0.252683,0.373839,0.252683,0.121156,0.065439,80303,-0.985699,-0.186662,...,-0.996865,-0.998628,-0.50,-0.807682,-0.970968,-0.825541,-0.737079,-0.400503,-0.866238,-1.0


# OLS ALL

In [8]:
# Complete number of observations is 680867
# We take the initial train, validate and oos sample to be 8, 5 and 13 years respectively (Ratios similar to Gu Kelly Xiu)
# On average one month contains 2000 options, therefore we loop over 12 * 2000 options, which is similar to looping over a year

# # Loop sizes
# trainsize = 8 * 3000 * 12
# validatesize = 5 * 3000 * 12
# oossize = 3000 * 12

# Loop sizes
trainsize = 11
validatesize = 5
oossize = 1

# Initialise predictionstring
IV = 'level'
oos_pred_level = pd.DataFrame()
oos_pred_slope = pd.DataFrame()
oos_pred_curve = pd.DataFrame()

for window in range(2012, 2021+1):
  print('Iteration: ', window)
  best = float("inf")

  # Sample splitting LEVEL
  IV = 'level'
  train, validate, oos, total_train, total_oos = samplesplitting(data, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly level:',r2_score(oos_y, y_hat))
  oos_pred_level = pd.concat([oos_pred_level, y_hat], ignore_index=True)
  print('Total level:',r2_score(total_oos[IV], oos_pred_level))

  # Sample splitting SLOPE
  IV = 'slope'
  train, validate, oos, total_train, total_oos = samplesplitting(data, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly slope:',r2_score(oos_y, y_hat))
  oos_pred_slope = pd.concat([oos_pred_slope, y_hat], ignore_index=True)
  print('Total slope:',r2_score(total_oos[IV], oos_pred_slope))

  # Sample splitting CURVE
  IV = 'curve'
  train, validate, oos, total_train, total_oos = samplesplitting(data, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly curve:',r2_score(oos_y, y_hat))
  oos_pred_curve = pd.concat([oos_pred_curve, y_hat], ignore_index=True)
  print('Total curve:',r2_score(total_oos[IV], oos_pred_curve))

oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/level_predictions_OLSAll_st_yearly.csv', index=True)
oos_pred_slope.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/slope_predictions_OLSAll_st_yearly.csv', index=True)
oos_pred_curve.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/curve_predictions_OLSAll_st_yearly.csv', index=True)

# oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/level_predictions_OLSAll_mp_yearly.csv', index=True)
# oos_pred_slope.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/slope_predictions_OLSAll_mp_yearly.csv', index=True)
# oos_pred_curve.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/curve_predictions_OLSAll_mp_yearly.csv', index=True)

Iteration:  2012
train=210839, validate=158189, oos=33864
Yearly level: 0.3654930241497434
Total level: 0.3654930241497434
train=210839, validate=158189, oos=33864
Yearly slope: -0.015442941858514025
Total slope: -0.015442941858514025
train=210839, validate=158189, oos=33864
Yearly curve: -0.03655715454808095
Total curve: -0.03655715454808095
Iteration:  2013
train=241135, validate=161757, oos=34182
Yearly level: 0.22168480981890304
Total level: 0.30148953183612115
train=241135, validate=161757, oos=34182
Yearly slope: 0.01550993793076072
Total slope: 0.003635972904047602
train=241135, validate=161757, oos=34182
Yearly curve: 0.00041403457891586726
Total curve: -0.01625490494017101
Iteration:  2014
train=272284, validate=164790, oos=34113
Yearly level: 0.30233898151997296
Total level: 0.30277176242134585
train=272284, validate=164790, oos=34113
Yearly slope: -0.034601231816748346
Total slope: -0.004455686401681369
train=272284, validate=164790, oos=34113
Yearly curve: -0.06859461825048

# Selected Features

### Download data with selectected features

In [ ]:
data_level = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_level_standardized_selection10.csv'))
data_slope = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_slope_standardized_selection10.csv'))
data_curve = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_curve_standardized_selection10.csv'))


In [ ]:
data_level = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_level_mapped_selection10.csv'))
data_slope = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_slope_mapped_selection10.csv'))
data_curve = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_curve_mapped_selection10.csv'))

### OLS data

In [ ]:
data_curve.head()

,secid,-80,-50,-20,level,slope,curve,permno,betasq,dolvol,...,cinvest,roavol,roeq,rsup,ms,baspread,maxret,retvol,std_dolvol,std_turn
date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,100862,0.307688,0.315627,0.358095,0.315627,0.042468,0.017264,54594,-0.886445,-0.777471,...,0.071798,-0.953113,0.281230,-0.105877,0.00,-0.689049,-0.662673,-0.619551,0.327234,-0.936622
1996-01-31,100871,0.705411,0.615432,0.651160,0.615432,0.035728,0.062854,50906,0.193703,0.751197,...,0.080067,-0.900059,0.325452,-0.155422,0.75,-0.112575,-0.390130,-0.223204,-0.342296,-0.748948
1996-01-31,100892,0.270090,0.216768,0.216508,0.216768,-0.000260,0.026531,57904,-0.902575,0.329181,...,0.066337,-0.998951,0.345889,0.027781,0.25,-0.786859,-0.947305,-0.877289,-0.536138,-0.955821
1996-01-31,100896,0.430796,0.436291,0.429517,0.436291,-0.006774,-0.006135,77520,-0.101800,0.355979,...,0.060739,-0.859083,0.397384,0.633026,-0.25,-0.747736,-0.649524,-0.703492,-0.331142,-0.818216
1996-01-31,100903,0.262405,0.252683,0.373839,0.252683,0.121156,0.065439,80303,-0.682642,0.211679,...,0.077826,-0.875346,0.584032,0.043750,-0.50,-0.807682,-0.825541,-0.737079,-0.400503,-0.866238


In [ ]:
data_curve.head()

,secid,-80,-50,-20,level,slope,curve,permno,betasq,dolvol,...,cinvest,roavol,roeq,rsup,ms,baspread,maxret,retvol,std_dolvol,std_turn
date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,100862,0.307688,0.315627,0.358095,0.315627,0.042468,0.017264,54594,-0.628458,-2.342547,...,0.090355,-0.491351,-0.291363,0.017690,-0.324628,-0.424243,0.383028,-0.139901,2.271301,-0.457693
1996-01-31,100871,0.705411,0.615432,0.651160,0.615432,0.035728,0.062854,50906,2.509349,1.178146,...,0.161215,-0.260014,-0.020269,-0.315352,1.735954,1.474995,1.612154,1.173917,0.136424,0.297064
1996-01-31,100892,0.270090,0.216768,0.216508,0.216768,-0.000260,0.026531,57904,-0.675317,0.206196,...,0.043550,-0.691224,0.105018,0.916132,0.362233,-0.746485,-0.900621,-0.994256,-0.481665,-0.534902
1996-01-31,100896,0.430796,0.436291,0.429517,0.436291,-0.006774,-0.006135,77520,1.650918,0.267915,...,-0.004423,-0.081339,0.420696,4.984565,-1.011489,-0.617592,0.442328,-0.418152,0.171988,0.018494
1996-01-31,100903,0.262405,0.252683,0.373839,0.252683,0.121156,0.065439,80303,-0.036415,-0.064423,...,0.142014,-0.152253,1.564896,1.023473,-1.698350,-0.815089,-0.351483,-0.529485,-0.049179,-0.174634


### OLS-30,40,40 with standardised data

In [9]:
data_level = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_level_standardized_selection10.csv'))
data_slope = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_slope_standardized_selection10.csv'))
data_curve = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_curve_standardized_selection10.csv'))

# # Loop sizes
# trainsize = 8 * 3000 * 12
# validatesize = 5 * 3000 * 12
# oossize = 3000 * 12

# Loop sizes
trainsize =  11
validatesize = 5
oossize = 1

# Initialise predictionstring
oos_pred_level = pd.DataFrame()
oos_pred_slope = pd.DataFrame()
oos_pred_curve = pd.DataFrame()

for window in range(2012, 2021+1):
  print('Iteration: ', window)
  best = float("inf")

  # Sample splitting LEVEL
  IV = 'level'
  train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)
  print(len(train), len(validate), len(oos), len(total_train), len(total_oos))
  total_oos_x, total_oos_y      = split_xy(total_oos, IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)
  print(model_final.coef_, total_train_x.columns.values.tolist())

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly level:',r2_score(oos_y, y_hat))
  oos_pred_level = pd.concat([oos_pred_level, y_hat], ignore_index=True)
  # print('Total level:',r2_score(total_oos[IV], oos_pred_level))
  print('Total level:',r2_score(total_oos_y, oos_pred_level))

  # Sample splitting SLOPE
  IV = 'slope'
  train, validate, oos, total_train, total_oos = samplesplitting(data_slope, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly slope:',r2_score(oos_y, y_hat))
  oos_pred_slope = pd.concat([oos_pred_slope, y_hat], ignore_index=True)
  print('Total slope:',r2_score(total_oos[IV], oos_pred_slope))

  # Sample splitting CURVE
  IV = 'curve'
  train, validate, oos, total_train, total_oos = samplesplitting(data_curve, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly curve:',r2_score(oos_y, y_hat))
  oos_pred_curve = pd.concat([oos_pred_curve, y_hat], ignore_index=True)
  print('Total curve:',r2_score(total_oos[IV], oos_pred_curve))

oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/level_predictions_OLS30_st_yearly.csv', index=True)
oos_pred_slope.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/slope_predictions_OLS40_st_yearly.csv', index=True)
oos_pred_curve.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/curve_predictions_OLS40_st_yearly.csv', index=True)


Iteration:  2012
train=210839, validate=158189, oos=33864
210839 158189 33864 369028 33864
[ 1.72221397e-02 -8.39126290e-03 -2.18201244e-02  4.23629537e-02
  2.13351287e-02  2.07933271e-03 -2.58110230e-03  1.88857955e-03
  3.29047154e-03  6.09392313e-04 -1.46014428e-03  6.68661605e-03
  2.66078168e-04 -2.65144611e-03  1.94853984e-03  2.10145352e-03
 -4.98442869e-03 -1.02027946e-03  5.69682863e-03 -3.17645513e-03
  1.11182223e-02 -6.54281447e-05 -9.30956089e-03  2.38632030e-03
 -1.05818395e-03  9.16981920e-02 -6.82859575e-03  1.67379619e-02
  2.25886281e-03 -1.22206889e-02] ['beta', 'betasq', 'dolvol', 'idiovol', 'turn', 'absacc', 'age', 'bm_ia', 'convind', 'dy', 'ep', 'mve_ia', 'orgcap', 'ps', 'rd', 'rd_mve', 'roic', 'secured', 'securedind', 'tang', 'cash', 'cinvest', 'roaq', 'roavol', 'ms', 'baspread', 'maxret', 'retvol', 'std_dolvol', 'std_turn']
Yearly level: 0.36166070709188547
Total level: 0.36166070709188547
train=210839, validate=158189, oos=33864
Yearly slope: -0.01783407147755

### OLS-30,40,40 with mapped data

In [10]:
# # Loop sizes
# trainsize = 8 * 3000 * 12
# validatesize = 5 * 3000 * 12
# oossize = 3000 * 12

data_level = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_level_mapped_selection10.csv'))
data_slope = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_slope_mapped_selection10.csv'))
data_curve = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_curve_mapped_selection10.csv'))

# Loop sizes
trainsize = 11
validatesize = 5
oossize = 1

# Initialise predictionstring
oos_pred_level = pd.DataFrame()
oos_pred_slope = pd.DataFrame()
oos_pred_curve = pd.DataFrame()

for window in range(2012, 2021+1):
  print('Iteration: ', window)
  best = float("inf")

  # Sample splitting LEVEL
  IV = 'level'
  train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)
  print(len(train), len(validate), len(oos), len(total_train), len(total_oos))
  total_oos_x, total_oos_y      = split_xy(total_oos, IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)
  print(model_final.coef_, total_train_x.columns.values.tolist())

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly level:',r2_score(oos_y, y_hat))
  oos_pred_level = pd.concat([oos_pred_level, y_hat], ignore_index=True)
  # print('Total level:',r2_score(total_oos[IV], oos_pred_level))
  print('Total level:',r2_score(total_oos_y, oos_pred_level))

  # Sample splitting SLOPE
  IV = 'slope'
  train, validate, oos, total_train, total_oos = samplesplitting(data_slope, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly slope:',r2_score(oos_y, y_hat))
  oos_pred_slope = pd.concat([oos_pred_slope, y_hat], ignore_index=True)
  print('Total slope:',r2_score(total_oos[IV], oos_pred_slope))

  # Sample splitting CURVE
  IV = 'curve'
  train, validate, oos, total_train, total_oos = samplesplitting(data_curve, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly curve:',r2_score(oos_y, y_hat))
  oos_pred_curve = pd.concat([oos_pred_curve, y_hat], ignore_index=True)
  print('Total curve:',r2_score(total_oos[IV], oos_pred_curve))

oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/level_predictions_OLS30_mp_yearly.csv', index=True)
oos_pred_slope.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/slope_predictions_OLS40_mp_yearly.csv', index=True)
oos_pred_curve.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/curve_predictions_OLS40_mp_yearly.csv', index=True)


Iteration:  2012
train=210839, validate=158189, oos=33864
210839 158189 33864 369028 33864
[-0.0343023   0.05150057 -0.02704873  0.20431636  0.03102563  0.00758864
 -0.00126121  0.09158587  0.00108161 -0.03914628  0.06019839 -0.01584863
  0.01777089 -0.06031628  0.00503959 -0.01102379 -0.00131488 -0.00407655
  0.00855185 -0.01170454  0.02243872  0.02754758 -0.00219903  0.01934132
 -0.01029732  0.36096483 -0.00254311  0.01989946  0.01321047  0.00137822] ['beta', 'betasq', 'dolvol', 'idiovol', 'turn', 'absacc', 'age', 'bm_ia', 'convind', 'dy', 'ep', 'mve_ia', 'orgcap', 'ps', 'rd', 'rd_mve', 'roic', 'secured', 'securedind', 'tang', 'cash', 'cinvest', 'roaq', 'roavol', 'ms', 'baspread', 'maxret', 'retvol', 'std_dolvol', 'std_turn']
Yearly level: 0.41500698171047345
Total level: 0.41500698171047345
train=210839, validate=158189, oos=33864
Yearly slope: 0.005886130116876109
Total slope: 0.005886130116876109
train=210839, validate=158189, oos=33864
Yearly curve: -0.005606506170369396
Total cu

# OLS - 3/4

### OLS - 4 regressors (market beta (beta), bid-ask spread (baspread), book to market ratio (bm of bm_ia), leverage(lev)) (industry adjusted)

In [12]:
data_mp = data.copy()

In [13]:
df = data_mp.copy()
selected_columns = df.iloc[:, :8]  # Selecting the first 8 columns
# additional_columns = df[['bm_ia', 'beta', 'baspread', 'lev']]  # Selecting additional specific columns
additional_columns = df[['bm_ia', 'beta', 'baspread', 'lev']]  # Selecting additional specific columns
# additional_columns = df[[ 'beta', 'baspread', 'lev']]  # Selecting additional specific columns
data = pd.concat([selected_columns, additional_columns], axis=1)
data.head()

,secid,-80,-50,-20,level,slope,curve,permno,bm_ia,beta,baspread,lev
date,,,,,,,,,,,,
1996-01-31,100862,0.307688,0.315627,0.358095,0.315627,0.042468,0.017264,54594,0.328068,-0.502126,-0.689049,-0.918140
1996-01-31,100871,0.705411,0.615432,0.651160,0.615432,0.035728,0.062854,50906,0.302852,0.551365,-0.112575,-0.995785
1996-01-31,100892,0.270090,0.216768,0.216508,0.216768,-0.000260,0.026531,57904,0.293655,-0.536730,-0.786859,-0.491185
1996-01-31,100896,0.430796,0.436291,0.429517,0.436291,-0.006774,-0.006135,77520,0.286002,0.349363,-0.747736,-0.821394
1996-01-31,100903,0.262405,0.252683,0.373839,0.252683,0.121156,0.065439,80303,0.171681,-0.186662,-0.807682,-0.838204


In [14]:
# Complete number of observations is 680867
# We take the initial train, validate and oos sample to be 8, 5 and 13 years respectively (Ratios similar to Gu Kelly Xiu)
# On average one month contains 2000 options, therefore we loop over 12 * 2000 options, which is similar to looping over a year

# # Loop sizes
# trainsize = 8 * 3000 * 12
# validatesize = 5 * 3000 * 12
# oossize = 3000 * 12

# Loop sizes
trainsize = 11
validatesize = 5
oossize = 1

# Initialise predictionstring
IV = 'level'
oos_pred_level = pd.DataFrame()
oos_pred_slope = pd.DataFrame()
oos_pred_curve = pd.DataFrame()

for window in range(2012, 2021+1):
  print('Iteration: ', window)
  best = float("inf")

  # Sample splitting LEVEL
  IV = 'level'
  train, validate, oos, total_train, total_oos = samplesplitting(data, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly level:',r2_score(oos_y, y_hat))
  oos_pred_level = pd.concat([oos_pred_level, y_hat], ignore_index=True)
  print('Total level:',r2_score(total_oos[IV], oos_pred_level))

  # Sample splitting SLOPE
  IV = 'slope'
  train, validate, oos, total_train, total_oos = samplesplitting(data, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly slope:',r2_score(oos_y, y_hat))
  oos_pred_slope = pd.concat([oos_pred_slope, y_hat], ignore_index=True)
  print('Total slope:',r2_score(total_oos[IV], oos_pred_slope))

  # Sample splitting CURVE
  IV = 'curve'
  train, validate, oos, total_train, total_oos = samplesplitting(data, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  # Fit model and produce forecasts
  model_final = LinearRegression()
  model_final.fit(total_train_x, total_train_y)

  # Compute predictions
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly curve:',r2_score(oos_y, y_hat))
  oos_pred_curve = pd.concat([oos_pred_curve, y_hat], ignore_index=True)
  print('Total curve:',r2_score(total_oos[IV], oos_pred_curve))

print('Total level:',r2_score(total_oos['level'], oos_pred_level))
print('Total slope:',r2_score(total_oos['slope'], oos_pred_slope))
print('Total curve:',r2_score(total_oos['curve'], oos_pred_curve))

oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/level_predictions_OLS4_mp_yearly.csv', index=True)
oos_pred_slope.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/slope_predictions_OLS4_mp_yearly.csv', index=True)
oos_pred_curve.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/OLS/curve_predictions_OLS4_mp_yearly.csv', index=True)

Iteration:  2012
train=210839, validate=158189, oos=33864
Yearly level: 0.3026905949529918
Total level: 0.3026905949529918
train=210839, validate=158189, oos=33864
Yearly slope: -0.0277214684751812
Total slope: -0.0277214684751812
train=210839, validate=158189, oos=33864
Yearly curve: -0.04609021704744687
Total curve: -0.04609021704744687
Iteration:  2013
train=241135, validate=161757, oos=34182
Yearly level: 0.19893465218711093
Total level: 0.25914191328324043
train=241135, validate=161757, oos=34182
Yearly slope: 0.0029426408533388404
Total slope: -0.008730453877434652
train=241135, validate=161757, oos=34182
Yearly curve: -0.008436194219447968
Total curve: -0.025437682712105003
Iteration:  2014
train=272284, validate=164790, oos=34113
Yearly level: 0.2932002220661072
Total level: 0.2720417575307774
train=272284, validate=164790, oos=34113
Yearly slope: -0.053017655522702656
Total slope: -0.01938836511283837
train=272284, validate=164790, oos=34113
Yearly curve: -0.08019361447294071


4 bm:
Total level: 0.340108358738355
Total slope: -0.023456188163347047
Total curve: -0.05388141545634917

4 (bm_ia): Total level: 0.34952603959697404
Total slope: -0.021262673217462114
Total curve: -0.05307616452170083

zonder lev: Total level: 0.33944472901101874
Total slope: -0.023441170891076668
Total curve: -0.05403927183921575

zonder bs: Total level: -0.010496210677015227
Total slope: -0.028609936612457254
Total curve: -0.05675022035882216

zonder beta:
Total level: 0.340108358738355
Total slope: -0.023456188163347047
Total curve: -0.05388141545634917